In [1]:
import random

import numpy as np
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api

In [2]:
# corpus = api.load('text8')  # download the corpus and return it opened as an iterable
# word2vec = Word2Vec(corpus, vector_size=100)  # train a model from the corpus

info = api.info()  # show info about available models/datasets
word2vec = api.load("glove-wiki-gigaword-100")

In [3]:
# word2vec = word2vec.wv
# wikigiga_model.save("word2vec/glove-wiki-gigaword-100.model")
word2vec.has_index_for("Nightmare")
# wikigiga_model.most_similar("nightmare")
word2vec.get_vector("hot")
# len(wikigiga_model.word_vec("hot"))

array([-1.0239   ,  0.31711  ,  0.32645  , -0.21064  ,  0.28183  ,
       -0.045844 ,  0.69035  ,  0.13584  , -0.28214  , -0.27415  ,
        0.43748  , -0.50725  , -0.023477 ,  0.38268  ,  0.25041  ,
       -0.42515  ,  0.048989 , -0.50624  ,  0.20183  , -0.20446  ,
        0.48532  ,  0.96227  ,  0.073216 , -0.20945  , -0.15535  ,
        1.1081   ,  0.34316  , -0.28191  , -0.16772  ,  0.21148  ,
        0.16443  ,  0.62912  ,  0.34025  ,  0.82183  , -0.30162  ,
        0.92553  , -0.28931  , -0.35684  ,  0.55665  , -0.57464  ,
        0.11891  , -0.6799   , -0.35478  , -0.99149  ,  0.60496  ,
       -0.023199 , -0.57014  , -0.04316  ,  0.74405  , -0.95329  ,
       -0.0031113, -0.10723  , -0.57443  ,  0.39808  , -0.053929 ,
       -2.492    ,  0.49922  ,  0.79235  ,  1.3588   ,  0.39502  ,
       -0.098789 ,  1.0618   , -0.2236   ,  0.26782  ,  1.0612   ,
        0.11189  ,  0.87032  , -0.84204  ,  0.27031  , -0.77449  ,
       -0.12172  , -0.28352  ,  0.043281 ,  1.2128   , -0.3973

In [6]:
import codecs
# read mapping from genre_id to genre_names
genre_dict = {}
f = open('data/ml-1m/formatted/genres.txt', 'r')
for line in f:
    line_seg = line.rstrip().split(' ')
    genre_dict[int(line_seg[0])] = line_seg[1] # int(gen_id) -> str(gen_type)

# read title and genre strings for movies
movies = {}
genres = {}
cs_items = []
f = codecs.open('data/ml-1m/formatted/movies.txt', 'r', encoding='iso-8859-15')
for line in f:
    line_seg = line.rstrip().split('::')
    id = int(line_seg[0])
    movies[id] = line_seg[1]
    gen_id_strs = line_seg[3].split(',')
    if len(gen_id_strs) == 0:
        gen_str = ''
    else:
        gen_str = genre_dict[int(gen_id_strs[0])]
        for i in range(1, len(gen_id_strs)):
            gen_str += (' ' + genre_dict[int(gen_id_strs[i])])
    genres[id] = gen_str

# read cold-start movies
f = open('data/ml-1m/splits/cs_item_list.txt', 'r')
for line in f:
    cs_items.append(int(line.rstrip()))

# read stopwords
stopwords = []
f = open('word2vec/stopwords_English.txt', 'r')
for line in f:
    stopwords.append(line.rstrip())

In [7]:
print(stopwords[3])
test_mid = 3431
print("%s (%s) -- cs:%s" % (movies[test_mid], genres[test_mid], test_mid in cs_items))

about
Bat, The (Horror) -- cs:True


In [15]:
def get_avg_vector(model, stopwords, text):
    valid_words = 0
    sum_vec = np.zeros(100)
    for word in text.lower().rstrip().split(' '):
        if word not in stopwords and model.has_index_for(word):
            valid_words += 1
            sum_vec += model.get_vector(word)
    return (sum_vec / valid_words) if valid_words > 0 else sum_vec

cs_movie_vectors = {}
ws_movie_vectors = {} # word vectors of all non-cold-start movies
                         # [avg of title word vectors : avg of genre vectors]
for mid in movies:
    if mid in cs_items: continue
    m_title = movies[mid]
    m_genres = genres[mid]
    ws_movie_vectors[mid] = get_avg_vector(word2vec, stopwords, m_genres)
    # ws_movie_vectors[mid] = np.concatenate(
    #     (get_avg_vector(word2vec, stopwords, m_title),
    #      get_avg_vector(word2vec, stopwords, m_genres)), axis=0)
for mid in cs_items:
    m_title = movies[mid]
    m_genres = genres[mid]
    cs_movie_vectors[mid] = get_avg_vector(word2vec, stopwords, m_genres)
    # cs_movie_vectors[mid] = np.concatenate(
    #     (get_avg_vector(word2vec, stopwords, m_title),
    #      get_avg_vector(word2vec, stopwords, m_genres)), axis=0)

In [16]:
similar_items = {} # mid -> [similar ids]
for csid in cs_items:
    cs_vector = cs_movie_vectors[csid]
    similarity = {}
    for wsid in ws_movie_vectors:
        similarity[wsid] = np.linalg.norm(cs_vector - ws_movie_vectors[wsid])
    similar_items[csid] = []
    for i in range(3):
        sid, _ = min(similarity.items(), key=lambda x: x[1])
        similar_items[csid].append(sid)
        similarity.pop(sid)

In [17]:
test_id = 3235
test_similar_ids = similar_items[test_id]
print("%d:%s (%s) -- cs:%s" % (test_id, movies[test_id], genres[test_id], test_id in cs_items))
print("Top similar items:")
for sid in test_similar_ids:
    print("%d:%s (%s) -- cs:%s" % (sid, movies[sid], genres[sid], sid in cs_items))
print(len(word2vec.get_vector('hot')))

3235:Alien Escape (Horror Sci-Fi) -- cs:True
Top similar items:
269:Faculty, The (Horror Sci-Fi) -- cs:False
295:Night of the Living Dead (Horror Sci-Fi) -- cs:False
949:Invisible Man, The (Horror Sci-Fi) -- cs:False
100


In [19]:
output_f = open('data/ml-1m/similar_items/wikigiga_100_genre.txt', 'w')
for csid in cs_items:
    synonyms_str = ''
    for sid in similar_items[csid]: synonyms_str += (' ' + str(sid))
    output_f.write("%s%s\n" % (csid, synonyms_str))
output_f.close()
print('Done')

Done


In [32]:
len(word2vec) # text8
2682 in list(ws_movie_vectors)

False

In [34]:
output_f = open('data/ml-1m/similar_items/random_con.txt', 'w')
for csid in cs_items:
    similar_items = random.sample(list(ws_movie_vectors), 3)
    synonyms_str = ''
    for sid in similar_items: synonyms_str += (' ' + str(sid))
    output_f.write("%s%s\n" % (csid, synonyms_str))
output_f.close()
print('Done')

Done
